In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
!pip install ipywidgets

In [ ]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

def calculate_costs(orders, start_bundles, prepaid_bundles, overage_cost):
    min_total_cost = float('inf')
    best_combination = None
    
    for starter_cost, starter_orders, starter_type in start_bundles:
        initial_bundle_cost = starter_cost
        initial_bundle_orders = starter_orders
        
        remaining_orders = max(0, orders - initial_bundle_orders)
        
        for num_prepaids in range((remaining_orders // min(bundle[1] for bundle in prepaid_bundles)) + 1):
            for bundle_combo in itertools.combinations_with_replacement(prepaid_bundles, num_prepaids):
                total_bundle_cost = sum(bundle_cost for bundle_cost, bundle_orders, bundle_type in bundle_combo)
                total_bundle_orders = sum(bundle_orders for bundle_cost, bundle_orders, bundle_type in bundle_combo)
                
                remaining_orders_after_bundles = remaining_orders - total_bundle_orders
                
                if remaining_orders_after_bundles > 0:
                    overage_cost_total = remaining_orders_after_bundles * overage_cost
                else:
                    overage_cost_total = 0
                    remaining_orders_after_bundles = 0
                
                total_cost = initial_bundle_cost + total_bundle_cost + overage_cost_total
                
                if total_cost < min_total_cost:
                    min_total_cost = total_cost
                    best_combination = (starter_cost, starter_orders, starter_type, bundle_combo, remaining_orders_after_bundles, overage_cost_total)
    
    return min_total_cost, best_combination

def bundle_description(bundle_combo):
    bundle_counts = {}
    for bundle_cost, bundle_orders, bundle_type in bundle_combo:
        if (bundle_cost, bundle_orders) in bundle_counts:
            bundle_counts[(bundle_cost, bundle_orders)] += 1
        else:
            bundle_counts[(bundle_cost, bundle_orders)] = 1
    
    descriptions = []
    for (bundle_cost, bundle_orders), count in bundle_counts.items():
        if count > 1:
            descriptions.append(f"{count}x (€{bundle_cost} for {bundle_orders} orders)")
        else:
            descriptions.append(f"€{bundle_cost} for {bundle_orders} orders")
    
    return ", ".join(descriptions)

def display_costs_df(orders, start_bundles, prepaid_bundles, overage_cost):
    total_cost, (starter_cost, starter_orders, starter_type, bundle_combo, remaining_orders_after_bundles, overage_cost_total) = calculate_costs(
        orders, start_bundles, prepaid_bundles, overage_cost
    )
    
    small_bundles_count = sum(1 for bundle_cost, bundle_orders, bundle_type in bundle_combo if bundle_type == 'small')
    big_bundles_count = sum(1 for bundle_cost, bundle_orders, bundle_type in bundle_combo if bundle_type == 'big')
    
    small_bundle_descriptions = bundle_description([bundle for bundle in bundle_combo if bundle[2] == 'small'])
    big_bundle_descriptions = bundle_description([bundle for bundle in bundle_combo if bundle[2] == 'big'])
    
    used_starter_bundles = {
        'small': False,
        'big': False
    }
    if starter_type == 'small':
        used_starter_bundles['small'] = True
    elif starter_type == 'big':
        used_starter_bundles['big'] = True

    small_starter_description = f"€{starter_cost} for {starter_orders} orders" if used_starter_bundles['small'] else "Not used"
    big_starter_description = f"€{starter_cost} for {starter_orders} orders" if used_starter_bundles['big'] else "Not used"
    
    overage_cost_str = f"€{overage_cost_total:.2f} for {remaining_orders_after_bundles} extra orders" if remaining_orders_after_bundles != 0 else f"€{overage_cost_total:.0f} for {remaining_orders_after_bundles} extra orders"
    
    data = {
        'Description': [
            'Total Orders',
            'Small Start Cost',
            'Big Start Cost',
            'Small Prepaid Count',
            'Big Prepaid Count',
            'Small Prepaid Cost',
            'Big Prepaid Cost',
            'Overage Orders',
            'Overage Cost',
            'Total Cost'
        ],
        'Value': [
            orders,
            small_starter_description,
            big_starter_description,
            small_bundles_count,
            big_bundles_count,
            small_bundle_descriptions,
            big_bundle_descriptions,
            remaining_orders_after_bundles,
            overage_cost_str,
            f"€{total_cost:.2f}"
        ]
    }
    
    df = pd.DataFrame(data)
    return df

def plot_costs(df, title):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('off')
    ax.axis('tight')
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(14)
    table.scale(1.5, 1.5)

    # Pas de rijhoogte aan
    for key, cell in table.get_celld().items():
        cell.set_edgecolor('black')
        cell.set_linewidth(2)
        cell.set_facecolor('white')
        cell.set_height(0.05)  # Verhoog de rijhoogte
    
    ax.set_title(title, fontweight='bold')
    plt.show()

# Interactieve widgets
orders_slider = widgets.IntSlider(value=5000, min=10, max=10000, step=10)
small_start_cost_slider = widgets.IntSlider(value=1000, min=500, max=2000, step=50)
small_start_orders_slider = widgets.IntSlider(value=100, min=50, max=500, step=10)
big_start_cost_slider = widgets.IntSlider(value=2000, min=1000, max=3000, step=50)
big_start_orders_slider = widgets.IntSlider(value=1350, min=1000, max=2000, step=50)
small_prepaid_cost_slider = widgets.IntSlider(value=250, min=100, max=500, step=50)
small_prepaid_orders_slider = widgets.IntSlider(value=250, min=100, max=500, step=50)
big_prepaid_cost_slider = widgets.IntSlider(value=1000, min=500, max=2000, step=100)
big_prepaid_orders_slider = widgets.IntSlider(value=1100, min=1000, max=1500, step=50)
overage_cost_slider = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1)

def update_plot(orders, small_start_cost, small_start_orders, big_start_cost, big_start_orders, small_prepaid_cost, small_prepaid_orders, big_prepaid_cost, big_prepaid_orders, overage_cost):
    start_bundles = [
        (small_start_cost, small_start_orders, 'small'),
        (big_start_cost, big_start_orders, 'big')
    ]
    
    prepaid_bundles = [
        (small_prepaid_cost, small_prepaid_orders, 'small'),
        (big_prepaid_cost, big_prepaid_orders, 'big')
    ]
    
    df = display_costs_df(orders, start_bundles, prepaid_bundles, overage_cost)
    plot_costs(df, "")

# Groepeer de sliders met labels erboven
sliders = widgets.VBox([
    widgets.HTML("<h2>Pricing Model Calculator</h2>"),
    widgets.VBox([widgets.Label(value='Orders:'), orders_slider]),
    widgets.VBox([widgets.Label(value='Small Start €:'), small_start_cost_slider]),
    widgets.VBox([widgets.Label(value='Small Start Orders:'), small_start_orders_slider]),
    widgets.VBox([widgets.Label(value='Big Start €:'), big_start_cost_slider]),
    widgets.VBox([widgets.Label(value='Big Start Orders:'), big_start_orders_slider]),
    widgets.VBox([widgets.Label(value='Small Prepaid €:'), small_prepaid_cost_slider]),
    widgets.VBox([widgets.Label(value='Small Prepaid Orders:'), small_prepaid_orders_slider]),
    widgets.VBox([widgets.Label(value='Big Prepaid €:'), big_prepaid_cost_slider]),
    widgets.VBox([widgets.Label(value='Big Prepaid Orders:'), big_prepaid_orders_slider]),
    widgets.VBox([widgets.Label(value='Overage €:'), overage_cost_slider])
])

interactive_plot = widgets.interactive_output(update_plot, {
    'orders': orders_slider,
    'small_start_cost': small_start_cost_slider,
    'small_start_orders': small_start_orders_slider,
    'big_start_cost': big_start_cost_slider,
    'big_start_orders': big_start_orders_slider,
    'small_prepaid_cost': small_prepaid_cost_slider,
    'small_prepaid_orders': small_prepaid_orders_slider,
    'big_prepaid_cost': big_prepaid_cost_slider,
    'big_prepaid_orders': big_prepaid_orders_slider,
    'overage_cost': overage_cost_slider
})

# CSS-stijl om de labels volledig zichtbaar te maken en sliders te centreren
display(HTML("<style>div.widget-label { min-width: 20ex !important; }</style>"))
display(widgets.VBox([sliders, interactive_plot]))